In [1]:
import torch
torch.cuda.empty_cache()

In [6]:

import random
import io
import os
import torch
import warnings
from tqdm.auto import tqdm
from torch.utils.data import Dataset, DataLoader
from ml_things import plot_dict, plot_confusion_matrix, fix_text
from sklearn.metrics import classification_report, accuracy_score
from transformers import (set_seed,
                          TrainingArguments,
                          Trainer,
                          GPT2Config,
                          GPT2Tokenizer,
                          AdamW, 
                          get_linear_schedule_with_warmup,
                          GPT2ForSequenceClassification)
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
warnings.filterwarnings('ignore')

In [3]:
set_seed(123)

# Number of training epochs (authors on fine-tuning Bert recommend between 2 and 4).
epochs = 4

# Number of batches - depending on the max sequence length and GPU memory.
# For 512 sequence length batch of 10 works without cuda memory issues.
# For small sequence length can try batch of 32 or higher.
batch_size = 32

# Pad or truncate text sequences to a specific length
# if `None` it will use maximum sequence of word piece tokens allowed by model.
max_length = 60

# Look for gpu to use. Will use `cpu` by default if no gpu found.
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


model_name_or_path='skt/kogpt2-base-v2'

# Dictionary of labels and their id - this will be used to convert.
# String labels to number ids.
labels_ids = {'sad':0,'sadness':0, 'anger':1,'angry':1,'disgust':2,'happiness':3,'fear':4,
              'neutral':5,'surprise':6}

# How many labels are we using in training.
# This is used to decide size of classification head.
n_labels = len(labels_ids)



In [4]:
import os

dir_path = "./dataset"
data_list = []
for (root, directories, files) in os.walk(dir_path):
    for file in files:
        print(file)
        if '.csv' in file:
            file_path = os.path.join(root, file)
            data_list.append(pd.read_csv(file_path,encoding="cp949"))
        if '.xlsx' in file:
            file_path = os.path.join(root, file)
            data_list.append(pd.read_excel(file_path))

training4.csv
training5.csv
training5_2.csv
training_korean.xlsx


In [12]:
neu_data_list = []
neu_data = data_list[3].iloc[:,1:]
neu_data.columns =  ["context","label"]
neu_data = neu_data.replace("중립","neutral")
neu_data = neu_data.replace("놀람","surprise")
neu_data = neu_data.replace("분노","angry")
neu_data = neu_data.replace("슬픔","sad")
neu_data = neu_data.replace("행복","happiness")

neu_index = list(neu_data[neu_data["label"]=="neutral"].index)
remove_index = random.sample(neu_index,33786)
neu_data = neu_data.drop(remove_index)


neu_data_list.append(neu_data[neu_data.label == "neutral"])
neu_data_list.append(neu_data[neu_data.label== "surprise"])
neu_data_list.append(neu_data[neu_data.label == "angry"])
neu_data_list.append(neu_data[neu_data.label == "sad"])
neu_data_list.append(neu_data[neu_data.label == "happiness"])

data = data_list[0].iloc[:,1:3]
for _ in range(1,3):
    data = pd.concat([data,data_list[_].iloc[:,1:3]])
data.columns = ["context","label"]
for buf in neu_data_list:
    data = pd.concat([data,buf])
data = data.replace("sadness","sad")
data = data.replace("anger","angry")

In [15]:
data.label.value_counts()

label
sad          15972
angry        15263
neutral      13262
surprise      6621
happiness     5578
disgust       4660
fear          4131
Name: count, dtype: int64

In [20]:

#encoder = LabelEncoder()
#data["label"] = encoder.fit_transform(data["label"])

In [21]:
train_x, val_x, train_y, val_y = train_test_split(data["context"],data["label"],test_size = 0.2,stratify=data["label"])

In [22]:
train_data = pd.DataFrame(columns=["context","label"])
train_data["context"]=train_x
train_data["label"]=train_y

In [54]:
labels_ids = {'sad': 0, 'angry': 1, 'disgust': 2, 'happiness': 3, 'fear': 4,
 'neutral': 5, 'surprise': 6}
# How many labels are we using in training.
# This is used to decide size of classification head.
n_labels = len(labels_ids)

import os

dir_path = "./dataset"
data_list = []
for (root, directories, files) in os.walk(dir_path):
    for file in files:
        if '.csv' in file:
            file_path = os.path.join(root, file)
            data_list.append(pd.read_csv(file_path,encoding="cp949"))
        if '.xlsx' in file:
            file_path = os.path.join(root, file)
            data_list.append(pd.read_excel(file_path))

data = data_list[0].iloc[:,1:3]

for _ in range(1,3):
    data = pd.concat([data,data_list[_].iloc[:,1:3]])
buf = data_list[3].iloc[:,1:]
#buf.columns = ['발화문',"a상황"]
#data = pd.concat([data,buf])
data.columns = ["context","label"]

data = data.replace("sad", "sadness")
data = data.replace("anger", "angry")

In [55]:
data.label.value_counts()

label
sadness      14000
angry        11635
disgust       4660
happiness     4548
fear          4131
neutral       3262
surprise      1755
Name: count, dtype: int64

In [57]:
data[data.label == "sadness" ]

,context,label
4,권태긴줄 알았는데 다른 사람이 생겼나보더라고.,sadness
5,그냥 걷고 있어.,sadness
7,처음 학원에서 만났다가 서로 좋아해서 사귀게 되었지.,sadness
8,내가 애정 표현을 잘 못해서 자주 싸우긴 했어.,sadness
9,오늘 헤어졌어.,sadness
...,...,...
19363,나 너무 우울해.,sadness
19364,밖에도 못나가고 집에서 일만 하니까 너무 힘들어.,sadness
19365,거의 안 움직이니까 너무 무기력한 것 같애.,sadness
19366,좋은 생각이야. 괜찮은 운동 있어?,sadness


In [33]:
class NewsDataset(Dataset):

    def __init__(self, data , use_tokenizer):

    # Check if path exists.
        """if not os.path.isdir(path):
          # Raise error if path is invalid.
          raise ValueError('Invalid `path` variable! Needs to be a directory')"""
        """self.texts = []
        self.labels = []

        # Since the labels are defined by folders with data we loop 
        # through each label.
        for label in ['ITscience', 'culture','economy','entertainment','health','life','politic','social','sport']:
          sentiment_path = os.path.join(path, label)

          # Get all files from path.
          files_names = os.listdir(sentiment_path)#[:10] # Sample for debugging.
          # Go through each file and read its content.
            for file_name in tqdm(files_names, desc=f'{label} files'):
                file_path = os.path.join(sentiment_path, file_name)

                # Read content.
                content = io.open(file_path, mode='r', encoding='utf-8').read()
                # Fix any unicode issues.
                content = fix_text(content)
                # Save content.
                self.texts.append(content)
                # Save encode labels.
                self.labels.append(label)
        """
        self.texts = np.array(data["context"])
        self.labels =np.array(data["label"])
        # Number of exmaples.
        self.n_examples = len(self.labels)


        return

    def __len__(self):
        r"""When used `len` return the number of examples.

        """
    
        return self.n_examples

    def __getitem__(self, item):
        r"""Given an index return an example from the position.

        Arguments:

          item (:obj:`int`):
              Index position to pick an example to return.

        Returns:
          :obj:`Dict[str, str]`: Dictionary of inputs that contain text and 
          asociated labels.

        """

        return {'text':self.texts[item],
                'label':self.labels[item]}



class Gpt2ClassificationCollator(object):
    r"""
    Data Collator used for GPT2 in a classificaiton rask. 
    
    It uses a given tokenizer and label encoder to convert any text and labels to numbers that 
    can go straight into a GPT2 model.

    This class is built with reusability in mind: it can be used as is as long
    as the `dataloader` outputs a batch in dictionary format that can be passed 
    straight into the model - `model(**batch)`.

    Arguments:

      use_tokenizer (:obj:`transformers.tokenization_?`):
          Transformer type tokenizer used to process raw text into numbers.

      labels_ids (:obj:`dict`):
          Dictionary to encode any labels names into numbers. Keys map to 
          labels names and Values map to number associated to those labels.

      max_sequence_len (:obj:`int`, `optional`)
          Value to indicate the maximum desired sequence to truncate or pad text
          sequences. If no value is passed it will used maximum sequence size
          supported by the tokenizer and model.

    """

    def __init__(self, use_tokenizer, labels_encoder, max_sequence_len=None):

        # Tokenizer to be used inside the class.
        self.use_tokenizer = use_tokenizer
        # Check max sequence length.
        self.max_sequence_len = use_tokenizer.model_max_length if max_sequence_len is None else max_sequence_len
        # Label encoder used inside the class.
        self.labels_encoder = labels_encoder

        return

    def __call__(self, sequences):
        r"""
        This function allowes the class objesct to be used as a function call.
        Sine the PyTorch DataLoader needs a collator function, I can use this 
        class as a function.

        Arguments:

          item (:obj:`list`):
              List of texts and labels.

        Returns:
          :obj:`Dict[str, object]`: Dictionary of inputs that feed into the model.
          It holddes the statement `model(**Returned Dictionary)`.
        """
        print(sequences)
        # Get all texts from sequences list.
        texts = [sequence['text'] for sequence in sequences]
        # Get all labels from sequences list.
        labels = [sequence['label'] for sequence in sequences]
        # Encode all labels using label encoder.
        labels = [self.labels_encoder[label] for label in labels]
        # Call tokenizer on all texts to convert into tensors of numbers with 
        # appropriate padding.
        inputs = self.use_tokenizer(text=texts, return_tensors="pt", padding=True, truncation=True,  max_length=self.max_sequence_len)
        # Update the inputs with the associated encoded labels as tensor.
        inputs.update({'labels':torch.tensor(labels)})

        return inputs







In [34]:
def train(dataloader, optimizer_, scheduler_, device_):
    """
      Train pytorch model on a single pass through the data loader.

      It will use the global variable `model` which is the transformer model 
      loaded on `_device` that we want to train on.

      This function is built with reusability in mind: it can be used as is as long
        as the `dataloader` outputs a batch in dictionary format that can be passed 
        straight into the model - `model(**batch)`.

      Arguments:

          dataloader (:obj:`torch.utils.data.dataloader.DataLoader`):
              Parsed data into batches of tensors.

          optimizer_ (:obj:`transformers.optimization.AdamW`):
              Optimizer used for training.

          scheduler_ (:obj:`torch.optim.lr_scheduler.LambdaLR`):
              PyTorch scheduler.

          device_ (:obj:`torch.device`):
              Device used to load tensors before feeding to model.

      Returns:

          :obj:`List[List[int], List[int], float]`: List of [True Labels, Predicted
            Labels, Train Average Loss].
    """

          # Use global variable for model.
    global model

          # Tracking variables.
    predictions_labels = []
    true_labels = []
          # Total loss for this epoch.
    total_loss = 0

          # Put the model into training mode.
    model.train()

      # For each batch of training data...
    for batch in tqdm(dataloader, total=len(dataloader)):

            # Add original labels - use later for evaluation.
        true_labels += batch['labels'].numpy().flatten().tolist()

            # move batch to device
        batch = {k:v.type(torch.long).to(device_) for k,v in batch.items()}

            # Always clear any previously calculated gradients before performing a
            # backward pass.
        model.zero_grad()

            # Perform a forward pass (evaluate the model on this training batch).
            # This will return the loss (rather than the model output) because we
            # have provided the `labels`.
            # The documentation for this a bert model function is here: 
            # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
        outputs = model(**batch)

            # The call to `model` always returns a tuple, so we need to pull the 
            # loss value out of the tuple along with the logits. We will use logits
            # later to calculate training accuracy.
        loss, logits = outputs[:2]

            # Accumulate the training loss over all of the batches so that we can
            # calculate the average loss at the end. `loss` is a Tensor containing a
            # single value; the `.item()` function just returns the Python value 
            # from the tensor.
        total_loss += loss.item()

            # Perform a backward pass to calculate the gradients.
        loss.backward()

            # Clip the norm of the gradients to 1.0.
            # This is to help prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

            # Update parameters and take a step using the computed gradient.
            # The optimizer dictates the "update rule"--how the parameters are
            # modified based on their gradients, the learning rate, etc.
        optimizer_.step()

            # Update the learning rate.
        scheduler_.step()

            # Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()

            # Convert these logits to list of predicted labels values.
        predictions_labels += logits.argmax(axis=-1).flatten().tolist()

      # Calculate the average loss over the training data.
    avg_epoch_loss = total_loss / len(dataloader)

      # Return all true labels and prediction for future evaluations.
    return true_labels, predictions_labels, avg_epoch_loss


In [35]:
def validation(dataloader, device_):
    """Validation function to evaluate model performance on a 
      separate set of data.

      This function will return the true and predicted labels so we can use later
      to evaluate the model's performance.

      This function is built with reusability in mind: it can be used as is as long
        as the `dataloader` outputs a batch in dictionary format that can be passed 
        straight into the model - `model(**batch)`.

      Arguments:

        dataloader (:obj:`torch.utils.data.dataloader.DataLoader`):
              Parsed data into batches of tensors.

        device_ (:obj:`torch.device`):
              Device used to load tensors before feeding to model.

      Returns:

        :obj:`List[List[int], List[int], float]`: List of [True Labels, Predicted
            Labels, Train Average Loss]
    """

  # Use global variable for model.
    global model

  # Tracking variables
    predictions_labels = []
    true_labels = []
      #total loss for this epoch.
    total_loss = 0

  # Put the model in evaluation mode--the dropout layers behave differently
  # during evaluation.
    model.eval()

  # Evaluate data for one epoch
    for batch in tqdm(dataloader, total=len(dataloader)):

        # add original labels
        true_labels += batch['labels'].numpy().flatten().tolist()

        # move batch to device
        batch = {k:v.type(torch.long).to(device_) for k,v in batch.items()}

        # Telling the model not to compute or store gradients, saving memory and
        # speeding up validation
        with torch.no_grad():        

            # Forward pass, calculate logit predictions.
            # This will return the logits rather than the loss because we have
            # not provided labels.
            # token_type_ids is the same as the "segment ids", which 
            # differentiates sentence 1 and 2 in 2-sentence tasks.
            # The documentation for this `model` function is here: 
            # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
            outputs = model(**batch)

            # The call to `model` always returns a tuple, so we need to pull the 
            # loss value out of the tuple along with the logits. We will use logits
            # later to to calculate training accuracy.
            loss, logits = outputs[:2]

            # Move logits and labels to CPU
            logits = logits.detach().cpu().numpy()

            # Accumulate the training loss over all of the batches so that we can
            # calculate the average loss at the end. `loss` is a Tensor containing a
            # single value; the `.item()` function just returns the Python value 
            # from the tensor.
            total_loss += loss.item()

            # get predicitons to list
            predict_content = logits.argmax(axis=-1).flatten().tolist()

            # update list
            predictions_labels += predict_content

  # Calculate the average loss over the training data.
    avg_epoch_loss = total_loss / len(dataloader)

  # Return all true labels and prediciton for future evaluations.
    return true_labels, predictions_labels, avg_epoch_loss
     

In [36]:
# Get model configuration.
print('Loading configuraiton...')
model_config = GPT2Config.from_pretrained(pretrained_model_name_or_path=model_name_or_path, num_labels=n_labels)

# Get model's tokenizer.
print('Loading tokenizer...')
from transformers import PreTrainedTokenizerFast

tokenizer = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2",
  bos_token='</s>', eos_token='</s>', unk_token='<unk>',
  pad_token='<pad>', mask_token='<mask>') 
# default to left padding
tokenizer.padding_side = "left"
# Define PAD Token = EOS Token = 50256
tokenizer.pad_token = tokenizer.eos_token


# Get the actual model.
print('Loading model...')
model = GPT2ForSequenceClassification.from_pretrained(pretrained_model_name_or_path=model_name_or_path, config=model_config)

# resize model embedding to match new tokenizer
model.resize_token_embeddings(len(tokenizer))

# fix model padding token id
model.config.pad_token_id = model.config.eos_token_id

# Load model to defined device.
model.to(device)
print('Model loaded to `%s`'%device)

Loading configuraiton...
Loading tokenizer...


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


Loading model...


Some weights of the model checkpoint at skt/kogpt2-base-v2 were not used when initializing GPT2ForSequenceClassification: ['lm_head.weight']
- This IS expected if you are initializing GPT2ForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing GPT2ForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at skt/kogpt2-base-v2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model loaded to `cuda`


In [37]:
# Create data collator to encode text and labels into numbers.
gpt2_classificaiton_collator = Gpt2ClassificationCollator(use_tokenizer=tokenizer, 
                                                          labels_encoder=labels_ids, 
                                                          max_sequence_len=max_length)


print('Dealing with Train...')
# Create pytorch dataset.
train_dataset = NewsDataset(data =data, 
                               use_tokenizer=tokenizer)
print('Created `train_dataset` with %d examples!'%len(train_dataset))

# Move pytorch dataset into dataloader.
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=gpt2_classificaiton_collator)
print('Created `train_dataloader` with %d batches!'%len(train_dataloader))

print()

#print('Dealing with Validation...')
# Create pytorch dataset.
#valid_dataset = NewsDataset(path='/content/drive/MyDrive/newsTopicClassification/sampling_test_data', 
#                               use_tokenizer=tokenizer)
#print('Created `valid_dataset` with %d examples!'%len(valid_dataset))

# Move pytorch dataset into dataloader.
#valid_dataloader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False, collate_fn=gpt2_classificaiton_collator)
#print('Created `eval_dataloader` with %d batches!'%len(valid_dataloader))

Dealing with Train...
Created `train_dataset` with 43991 examples!
Created `train_dataloader` with 1375 batches!



In [38]:
# Note: AdamW is a class from the huggingface library (as opposed to pytorch) 
# I believe the 'W' stands for 'Weight Decay fix"
optimizer = AdamW(model.parameters(),
                  lr = 2e-5, # default is 5e-5, our notebook had 2e-5
                  eps = 1e-8 # default is 1e-8.
                  )

# Total number of training steps is number of batches * number of epochs.
# `train_dataloader` contains batched data so `len(train_dataloader)` gives 
# us the number of batches.
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)

# Store the average loss after each epoch so we can plot them.
all_loss = {'train_loss':[], 'val_loss':[]}
all_acc = {'train_acc':[], 'val_acc':[]}

# Loop through each epoch.
print('Epoch')
for epoch in tqdm(range(epochs)):
    print()
    print('Training on batches...')
    # Perform one full pass over the training set.
    train_labels, train_predict, train_loss = train(train_dataloader, optimizer, scheduler, device)
    train_acc = accuracy_score(train_labels, train_predict)

  # Get prediction form model on validation data. 
    print('Validation on batches...')
    #valid_labels, valid_predict, val_loss = validation(valid_dataloader, device)
    #val_acc = accuracy_score(valid_labels, valid_predict)

  # Print loss and accuracy values to see how training evolves.
    #print("  train_loss: %.5f - val_loss: %.5f - train_acc: %.5f - valid_acc: %.5f"%(train_loss, val_loss, train_acc, val_acc))
    print("train_loss: {0:.5f} - train_acc: {1:.5f}".format(train_loss, train_acc))
    print()

  # Store the loss value for plotting the learning curve.
    all_loss['train_loss'].append(train_loss)
    #all_loss['val_loss'].append(val_loss)
    all_acc['train_acc'].append(train_acc)
    #all_acc['val_acc'].append(val_acc)



Epoch


  0%|          | 0/4 [00:00<?, ?it/s]


Training on batches...


  0%|          | 0/1375 [00:00<?, ?it/s]

[{'text': '나 다친 것 같아.', 'label': 'fear'}, {'text': '이야기 당연히 해봤지.', 'label': 'anger'}, {'text': '식당에서 밥을 먹는데 지진이 난거야.', 'label': 'fear'}, {'text': '어 그냥 우리집이나 친구집으로 옮길까? 전염병 빨리 끝났으면 좋겠어.', 'label': 'fear'}, {'text': '아니. 아무리 그래도 청소를 하면은 좀 사라져야 되는 거 아니야? 아우 짜증나 죽겠어.', 'label': 'angry'}, {'text': '나 내일 약속이 있는데 그대로 가도 될까?', 'label': 'fear'}, {'text': '해피가 어제 하늘 나라로 갔어. 집에 가기 싫어.', 'label': 'sad'}, {'text': '좋은 운동 있으면 추천해줘.', 'label': 'sadness'}, {'text': '지금은 괜찮으신데 또 다치실까봐 걱정이야.', 'label': 'anger'}, {'text': '맞아. 밖에 나가서 활동을 못 해서 그래.', 'label': 'sadness'}, {'text': '별로. 다 개인주의야.', 'label': 'anger'}, {'text': '3년 전에 친구 소개로 만나게 되었어.', 'label': 'sad'}, {'text': '상사한테 기획안이 잘못되었다고 많이 혼났어.', 'label': 'sad'}, {'text': '식당에 있던 사람들이 엎어지고 내 친구는 발목을 다쳤어!', 'label': 'fear'}, {'text': '약속 있어서 밖에 나가는데 갑자기 엘레베이터가 멈췄어.', 'label': 'fear'}, {'text': '응. 주의를 한 번 줬지. 그런데 하는 거 같더니만 다시 원래대로 지저분하게 쓰고 있어.', 'label': 'anger'}, {'text': '너무 무서웠지. 대처 요령을 공부 해둬야 될 거 같애. 같이 있었던 사람들은 크게 다치지는 않았어. 다행이야.', 'label': 'fear'

...

[{'text': '선물은 당연히 마음에 들지. 아주 비싼 향수야!', 'label': 'happiness'}, {'text': '냉장고 안에 약간의 반찬이 있는데 2주를 버티기에는 턱없이 부족해. 정말 어떻게 살지 앞날이 깜깜해!', 'label': 'sad'}, {'text': '나 오늘 짭새가 몰려 있길래 궁금해서 구경 갔다 왔어.', 'label': 'neutral'}, {'text': '고마워. 너도 무슨 일 생기면 바로 병원으로 가야 돼?', 'label': 'fear'}, {'text': '그럴려고. 방향제 지금 준비해놓고 청소 중이야.', 'label': 'disgust'}, {'text': '나갈 수가 없어. 밖에 물이 너무 많이 차올라있어.', 'label': 'fear'}, {'text': '힘들었는데 정말 행복했어. ', 'label': 'happiness'}, {'text': '해피가 엄청 뛰어다녀서 놀랐어. 하지만 다친 사람은 없어서 다행이야.', 'label': 'surprise'}, {'text': '모든 일처리가 비대면으로 이루어지니까 사람도 못 만나고 외출할 일도 없게 돼서 몸이 답답하고 처지게 돼.', 'label': 'sadness'}, {'text': '응! 날씨 완전 좋았어!', 'label': 'neutral'}, {'text': '말은 해봤지. 말은 했는데 만날때마다 얘긴 해봤지만 고쳐지지가 않아.', 'label': 'anger'}, {'text': '이번 회사가 공채 마지막 회사야. 떨어지면 인제 갈 데가 없다고.', 'label': 'sad'}, {'text': '맞아. 근데 이번에 신기록 달성했어.', 'label': 'happiness'}, {'text': '지금이 공채 시즌 마지막 인터라, 또 다른 데 어딜 지원해야 될지도 모르겠어.', 'label': 'sad'}, {'text': '하늘나라로 갔겠지? 착한아이였으니까.', 'label': 'sad'}, {'text': '전에 먹었던 음식물쓰레기 안치워서 

[{'text': '나 너무 우울해.', 'label': 'sadness'}, {'text': '내 선물이 뭔지 궁금해? 보고싶어? 나중에 꼭 보여줄게!', 'label': 'happiness'}, {'text': '그래. 기분이 좋아질 만 한 음악 들으면서 스트레스 풀고 싶다.', 'label': 'sadness'}, {'text': '나도 그럴려고 생각 중이야. 집에서 할게 없어.', 'label': 'angry'}, {'text': '기획안 작성 때문에 야근해야 될 거 같아.', 'label': 'sad'}, {'text': '보고서 다시 쓰려면 오늘도 밤 새워야할 것 같아.', 'label': 'sad'}, {'text': '해피가 어제 세상을 떠났어.', 'label': 'sad'}, {'text': '어. 다른 층을 써야긴 써야겠어. 많이 놀랐고, 그리고.', 'label': 'disgust'}, {'text': '카톡 엄청 보내놨지. 그런데 답장이 없어.', 'label': 'anger'}, {'text': '룸메이트랑 자꾸 싸우게 돼.', 'label': 'anger'}, {'text': '단톡방에 있는 지인이 시작한다고 하더라고.', 'label': 'sadness'}, {'text': '다들 무서웠는데 지금은 괜찮아졌어.', 'label': 'fear'}, {'text': '코로나 때문에 나가지도 못하고.', 'label': 'sadness'}, {'text': '고등학교 동창이었어 그때는 정말 순진하고 예쁜 아이이였어.', 'label': 'anger'}, {'text': '방향제 뿌렸다가 쓰레기 냄새랑 섞일까봐서.', 'label': 'disgust'}, {'text': '퇴근한지 얼마나 됐다고 회사에서 또 전화왔어.', 'label': 'anger'}, {'text': '그런게 있으면 지금 당장 구매해야지.', 'label': 'angry'}, {'text': '다치진 않고 다들 다 무사해.', 'label': 'fear'}, {'tex

[{'text': '아직. 부모님께 죄송해가지고 말씀도 못 드리겠어.', 'label': 'sad'}, {'text': '아침 저녁으로 환기해도 그렇다니까?', 'label': 'angry'}, {'text': '이번 회사가 공채로 보는 마지막 시험이었어. 다른 회사가 없어.', 'label': 'sad'}, {'text': '나 아까 해피랑 산책하다 정말 깜짝 놀랐어.', 'label': 'surprise'}, {'text': '우리 아빠 어제 또 다쳤어. ', 'label': 'anger'}, {'text': '단톡방에서 아는 지인이 주식을 추천해줘서 시작하게 됐어. ', 'label': 'sadness'}, {'text': '그냥 요즘 프로젝트도 잘 안되고. 약간 슬럼프 온 것 같은데. 너무 힘이 빠지고 아무 의욕도 없어. 아, 우울해.', 'label': 'sad'}, {'text': '응. 아직 가슴이 두근두근 거려.', 'label': 'fear'}, {'text': '해피가 어제 하늘로, 하늘 나라로 떠났어.', 'label': 'sad'}, {'text': '고마워. 그래도 내 잘못이 있으니까 혼났겠지. 내 잘못이 없었으면 혼나지도 않았을거야.', 'label': 'sad'}, {'text': '이미 회사 옥상에서 바람 쐬고 왔지.', 'label': 'sad'}, {'text': '여행도 너무 가고 싶다. 안 가본지 오래되가지고. ', 'label': 'happiness'}, {'text': '전화해서 그거 환불해달라고 했어. 사진도 찍고. ', 'label': 'disgust'}, {'text': '보지도 못하게 해서 바로 나와버렸어.', 'label': 'neutral'}, {'text': '한 일 주일 쯤? 된 거 같애.', 'label': 'sad'}, {'text': '매번 얘기를 해봤지! 그래도 고쳐지질 않아.', 'label': 'anger'}, {'text': '나 면접 또 떨어졌어.', 'label': 'sad'}, {'text

[{'text': '구급차도 온 거 보니까 크게 다친 것 같던데?', 'label': 'neutral'}, {'text': '응. 우리 고등학교 동창이었어.', 'label': 'anger'}, {'text': '어. 진짜 놀랐어. 다른 층 가봐야겠어.', 'label': 'disgust'}, {'text': '음식점 많기로 소문난 강남역이야.', 'label': 'neutral'}, {'text': '주식 가격이 폭락했어.', 'label': 'sadness'}, {'text': '우리 아빠가 어제 또 다치셨어.', 'label': 'anger'}, {'text': '그 방법도 한번 써 봐야 겠다.', 'label': 'angry'}, {'text': '이번 시즌 내가 원하던 마지막 회사였는데 떨어져서 또 면접 보는 게 의미가 있나 모르겠네.', 'label': 'sad'}, {'text': '좀 무섭긴 했어. 그 사람 유리병 들고있었거든.', 'label': 'neutral'}, {'text': '마라톤 대회에서 기존의 기록을 갱신했어.', 'label': 'happiness'}, {'text': '응급실 다녀와서 조금 나아지는 듯 하더니 갑자기 정신을 잃고 세상을 떠났어.', 'label': 'sad'}, {'text': '함께 달리는 크루가 있어서 서로 응원하면서 격려해줘서 너무나 힘이 돼.', 'label': 'happiness'}, {'text': '마라톤 대회에 출전했잖아. 내 기록이 엄청 좋게 나왔더라고.', 'label': 'happiness'}, {'text': '이번 면접에 또 떨어졌으니까 아무래도 다른 회사 갈 준비를 해야 해. 그런데 자꾸 면접에서 떨어지니까 자신감이 없어. 잘 될지 모르겠어.', 'label': 'sad'}, {'text': '집에서 일만 하니까 너무 더 힘든 것 같아.', 'label': 'sadness'}, {'text': '조금은 무기력해. 근데 많이 도움이 된다.', 'label': 'sadness'}, {'te

KeyboardInterrupt: 

In [ ]:
# Plot loss curves.
plot_dict(all_loss, use_xlabel='Epochs', use_ylabel='Value', use_linestyles=['-', '--'])

# Plot accuracy curves.
plot_dict(all_acc, use_xlabel='Epochs', use_ylabel='Value', use_linestyles=['-', '--'])

In [ ]:
!conda list

In [ ]:
 torch.cuda.is_available() 

In [ ]:
torch.cuda.get_device_name(0) 

In [ ]:
print(device)

In [ ]:

print('Device:', device)
print('가능',torch.cuda.device_count())
print('Current cuda device:', torch.cuda.current_device())
print('Count of using GPUs:', torch.cuda.device_count())
print(torch.cuda.get_device_name())

In [ ]:
print(torch.rand(5,4,device = 'cuda'))